# Do the PCA Amplitudes Correspond to Any Sensor

In [ ]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from astropy.constants import c
from astropy.timeseries import LombScargle
from scipy import interpolate
from scipy.stats import spearmanr, pearsonr
import pickle
from mpfit import mpfit

from tqdm import tqdm
import seaborn as sns

from waveCal import *

## Gather Files

In [ ]:
# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)
print(num_lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
# ThAr
thar_files = glob('/mnt/home/lzhao/ceph/thar5*/ThAr_*.fits')
thid_files  = glob('/mnt/home/lzhao/ceph/thid5*/ThAr_*.thid')
thar_files, thar_times = sortFiles(thar_files, get_mjd=True)
thar_files = thar_files[1:] # First file is from before LFCs
thar_times = thar_times[1:]
thid_files = sortFiles(thid_files) [1:]
num_thar_files = len(thar_files)
print(num_thar_files)

## Get and Save Patch Dictionaries

In [ ]:
ckpt_patch = pickle.load(open('./191204_ckptPatch9.pkl','rb'))
thid_patch = pickle.load(open('./191204_thidPatch15.pkl','rb'))

In [ ]:
sensors = pickle.load(open('./191218_sensors.pkl','rb'))

## Plot PCA Amplitudes and Sensor Values Over Time

In [ ]:
colors = sns.color_palette('plasma',len(sensors.keys()))

for i, key in enumerate(sensors.keys()):
    time, valu = sensors[key]
    mask = np.logical_and(valu!=0,valu!=-999.)
    if len(valu[mask])==0:
        continue
    max_v = np.median(np.sort(valu[mask])[-1000:])
    min_v = np.median(np.sort(valu[mask])[:1000])
    norm_v = (valu-min_v)/abs(max_v-min_v)
    
    ckpt_valu = np.interp(ckpt_patch['times'],time[mask],valu[mask],
                          left=np.nan,right=np.nan)
    nan_mask = ~np.isnan(ckpt_valu)
    spear0 = spearmanr(ckpt_patch['ec'][:,0][nan_mask],ckpt_valu[nan_mask])[0]
    pear0  = pearsonr(ckpt_patch['ec'][:,0][nan_mask],ckpt_valu[nan_mask])[0]
    spear1 = spearmanr(ckpt_patch['ec'][:,1][nan_mask],ckpt_valu[nan_mask])[0]
    pear1  = pearsonr(ckpt_patch['ec'][:,1][nan_mask],ckpt_valu[nan_mask])[0]
    
    plt.figure(figsize=(6.4*2,4.8))
    plt.subplot(121)
    plt.title('{}, $\\rho$={:.3}, r={:.3}'.format(key,spear0,pear0))
    plt.xlabel('PCA Amplitude 0')
    plt.ylabel(f'{key}')
    plt.plot(ckpt_patch['ec'][:,0],ckpt_valu,'.',color=colors[i])
    
    plt.subplot(122)
    plt.title('$\\rho$={:.3}, r={:.3}'.format(key,spear1,pear1))
    plt.xlabel('PCA Amplitude 1')
    plt.ylabel(f'{key}')
    plt.plot(ckpt_patch['ec'][:,1],ckpt_valu,'.',color=colors[i])
    
    plt.tight_layout()
    plt.savefig('./Figures/191219_corr{}.png'.format(key.replace(' ','')))

In [ ]:
colors = sns.color_palette('plasma',len(sensors.keys()))

for i, key in enumerate(sensors.keys()):
    time, valu = sensors[key]
    mask = np.logical_and(valu!=0,valu!=-999.)
    if len(valu[mask])==0:
        continue
    max_v = np.median(np.sort(valu[mask])[-1000:])
    min_v = np.median(np.sort(valu[mask])[:1000])
    norm_v = (valu-min_v)/abs(max_v-min_v)
    
    ckpt_valu = np.interp(ckpt_patch['times'],time[mask],valu[mask],
                          left=np.nan,right=np.nan)
    nan_mask = ~np.isnan(ckpt_valu)
    spear0 = spearmanr(ckpt_patch['ec'][:,0][nan_mask],ckpt_valu[nan_mask])[0]
    pear0  = pearsonr(ckpt_patch['ec'][:,0][nan_mask],ckpt_valu[nan_mask])[0]
    spear1 = spearmanr(ckpt_patch['ec'][:,1][nan_mask],ckpt_valu[nan_mask])[0]
    pear1  = pearsonr(ckpt_patch['ec'][:,1][nan_mask],ckpt_valu[nan_mask])[0]
    
    plt.figure(figsize=(6.4*2,4.8))
    plt.subplot(121)
    plt.title('{}, $\\rho$={:.3}, r={:.3}'.format(key,spear0,pear0))
    plt.xlabel('Time [mjd]')
    plt.ylabel('Normalized Values')
    y = ckpt_patch['ec'][:,0]
    norm_y = y/max(abs(y))
    plt.plot(time[mask],norm_v[mask],color=colors[i],label=key)
    plt.plot(ckpt_patch['times'],norm_y,'.-',label='PCA Amplitudes')
    plt.xlim(ckpt_patch['times'].min()-5,ckpt_patch['times'].max()+5)
    plt.ylim(-1.2,1.2)
    ax = plt.gca()
    ax.minorticks_on()
    ax.grid()
    ax.grid(which='minor',linestyle=':')
    
    plt.subplot(122)
    plt.title('PCA Component 2, $\\rho$={:.3}, r={:.3}'.format(spear1,pear1))
    plt.xlabel('Time [mjd]')
    plt.ylabel('Normalized Values')
    y = ckpt_patch['ec'][:,1]
    norm_y = y/max(abs(y))
    plt.plot(time[mask],norm_v[mask],color=colors[i],label=key)
    plt.plot(ckpt_patch['times'],norm_y,'.-',label='PCA Amplitudes')
    plt.xlim(ckpt_patch['times'].min()-5,ckpt_patch['times'].max()+5)
    plt.ylim(-1.2,1.2)
    ax = plt.gca()
    ax.minorticks_on()
    ax.grid()
    ax.grid(which='minor',linestyle=':')
    plt.tight_layout()
    plt.savefig('./Figures/191219_pcaaV{}.png'.format(key.replace(' ','')))

## Periodograms

In [ ]:
ls0 = LombScargle(ckpt_patch['times'],ckpt_patch['ec'][:,0])
ls1 = LombScargle(ckpt_patch['times'],ckpt_patch['ec'][:,1])
pergms = {}
per_plts = {}
sigs = {}
for key in tqdm(sensors.keys()):
    time, valu = sensors[key]
    mask = np.logical_and(valu!=0,valu!=-999.)
    if len(valu[mask])==0:
        continue
    pergms[key] = LombScargle(time[mask],valu[mask])
    per_plts[key] = pergms[key].autopower()
    sigs[key] = pergms[key].false_alarm_level([0.1,0.01])

In [ ]:
f0,p0 = ls0.autopower()
f1,p1 = ls1.autopower()

In [ ]:
plt.plot(1/f0,p0)
plt.plot(1/f1,p1)

for i,key in enumerate(per_plts.keys()):
    f,p = per_plts[key]
    plt.plot(1/f,p,color=colors[i],alpha=0.1)

plt.gca().set_xscale('log')

In [ ]:
f,p = ls0.autopower()
per_mask = np.logical_and((1/f) > 2.5/60/24, (1/f) < 80)
plt.figure()
plt.title('PCA Component 1')
plt.xlabel('Period [days]')
plt.ylabel('Power')
plt.plot(1/f[per_mask],p[per_mask])
sigm = ls0.false_alarm_level([0.1,0.01])
plt.axhline(sigm[0],color='k',linestyle='--',lw=1)
plt.axhline(sigm[1],color='k',linestyle='-',lw=1)
plt.axvline(1,color='k',alpha=0.5,lw=1)
plt.gca().set_xscale('log')
plt.xlim(2.5/60/24,80)
_,ymax = plt.ylim()
plt.ylim(0,ymax)
ax = plt.gca()
ax.minorticks_on()
ax.grid()
ax.grid(which='minor',linestyle=':')
plt.tight_layout()
plt.savefig('./Figures/191219_perio1.png')

In [ ]:
f,p = ls1.autopower()
per_mask = np.logical_and((1/f) > 2.5/60/24, (1/f) < 80)
plt.figure()
plt.title('PCA Component 2')
plt.xlabel('Period [days]')
plt.ylabel('Power')
plt.plot(1/f[per_mask],p[per_mask])
sigm = ls1.false_alarm_level([0.1,0.01])
plt.axhline(sigm[0],color='k',linestyle='--',lw=1)
plt.axhline(sigm[1],color='k',linestyle='-',lw=1)
plt.axvline(1,color='k',alpha=0.5,lw=1)
plt.gca().set_xscale('log')
plt.xlim(2.5/60/24,80)
_,ymax = plt.ylim()
plt.ylim(0,ymax)
ax = plt.gca()
ax.minorticks_on()
ax.grid()
ax.grid(which='minor',linestyle=':')
plt.tight_layout()
plt.savefig('./Figures/191219_perio2.png')

In [ ]:
for i,key in enumerate(per_plts.keys()):
    f,p = per_plts[key]
    per_mask = np.logical_and((1/f) > 2.5/60/24, (1/f) < 80)
    plt.figure()
    plt.title(key)
    plt.xlabel('Period [days]')
    plt.ylabel('Power')
    plt.plot(1/f[per_mask],p[per_mask],color=colors[i])
    sigm = sigs[key]
    plt.axhline(sigm[0],color='k',linestyle='--',lw=1)
    plt.axhline(sigm[1],color='k',linestyle='-',lw=1)
    plt.axvline(1,color='k',alpha=0.5,lw=1)
    plt.gca().set_xscale('log')
    plt.xlim(2.5/60/24,80)
    ax = plt.gca()
    ax.minorticks_on()
    ax.grid()
    ax.grid(which='minor',linestyle=':')
    plt.tight_layout()
    plt.savefig('./Figures/191219_perio{}.png'.format(key.replace(' ','')))